# 2D Estimator

## Imports and Setup

In [13]:
import os
import src.bug_dataset
from src.lifting_network import LiftingNet
from src.train_network import TrainNetwork
import numpy as np
import pandas as pd
from skimage import io
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [14]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    if len(points) < 62:
        return
    elif len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [15]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))

In [16]:
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir,transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))

KeyboardInterrupt: 

In [ ]:
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,valid_dataset ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

In [ ]:
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
reduceKeypoints = False
if reduceKeypoints:
    model = LiftingNet(2*29,1392,29)
else:
    model = LiftingNet(2*62,512,62)

accurate_dist = torch.from_numpy(np.nan_to_num(abs(([4,4,4]-sungaya_dataset.means_3d)/sungaya_dataset.std_3d)).T[2])
epochs = 10
learning_rate = 2.5e-1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
liftingtrainer = TrainNetwork(model,optimizer,accurate_dist,train_dataloader, valid_dataloader, reduceKeypoints)
trained_model = liftingtrainer.run(epochs)

C:\Users\JONATH~1\AppData\Local\Temp/ipykernel_12552/2031898071.py:7: RuntimeWarning: divide by zero encountered in true_divide
  accurate_dist = torch.from_numpy(np.nan_to_num(abs(([4,4,4]-sungaya_dataset.means_3d)/sungaya_dataset.std_3d)).T[2])


Using cuda device
Epoch 1
-------------------------------
loss: 0.382095  [    0/ 2286]
Validation Error: 
 Accuracy: 49.322086%, Avg loss: 0.222989 

Epoch 2
-------------------------------
loss: 0.217919  [    0/ 2286]
Validation Error: 
 Accuracy: 51.116564%, Avg loss: 0.187824 

Epoch 3
-------------------------------
loss: 0.181807  [    0/ 2286]
Validation Error: 
 Accuracy: 52.220859%, Avg loss: 0.166698 

Epoch 4
-------------------------------
loss: 0.160114  [    0/ 2286]
Validation Error: 
 Accuracy: 52.969325%, Avg loss: 0.152588 

Epoch 5
-------------------------------
loss: 0.145464  [    0/ 2286]
Validation Error: 
 Accuracy: 53.570552%, Avg loss: 0.142622 

Epoch 6
-------------------------------
loss: 0.134975  [    0/ 2286]
Validation Error: 
 Accuracy: 53.969325%, Avg loss: 0.135286 

Epoch 7
-------------------------------
loss: 0.127150  [    0/ 2286]
Validation Error: 
 Accuracy: 54.254601%, Avg loss: 0.129709 

Epoch 8
-------------------------------
loss: 0.121

In [17]:
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader:
        image = data['file_name']
        X = data['key_points_2D']
        y = data['key_points_3D']
        vis = data['visibility']
        # print(y)
        X, y = X.to(liftingtrainer.device, dtype=torch.float), y.to(liftingtrainer.device, dtype=torch.float)
        pred = trained_model(X)
        break


In [18]:
sample = 0
prediction = (pred[sample].cpu()).numpy()
actual = (y[sample].cpu()).numpy()
visibility_mask = vis[sample]

mean = sungaya_dataset.means_3d[:,2]
std = sungaya_dataset.std_3d[:,2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

unnormalised_acc = (actual*sungaya_dataset.std_3d)+sungaya_dataset.means_3d
ax, line1 = plot_stick_bug(ax, unnormalised_acc, visibility_mask)

unnormalised_est = unnormalised_acc
unnormalised_est[:,2] = (prediction*std)+mean
ax, line2 = plot_stick_bug(ax, unnormalised_acc, visibility_mask, True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.title(image[sample])
plt.legend(handles=[line1,line2], loc='upper right')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)
# opens external plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …